In [1]:
# Import required libraries
import numpy as np
import pandas as pd

import openmeteo_requests
import requests_cache
from retry_requests import retry

import matplotlib.pyplot as plt
import seaborn as sns

## Use command "pip freeze > requirements.txt" to save the dependencies in requirements.txt

### Open-Meteo Weather API - Doku
https://github.com/open-meteo/open-meteo?tab=readme-ov-file

Historical Weather: https://open-meteo.com/en/docs/historical-weather-api

Weather Forecast:   https://open-meteo.com/en/docs

Warning: 'Hourly API request limit exceeded. Please try again in the next hour.

### OpenStreetMap - Doku
https://www.openstreetmap.org/#map=8/46.826/8.613

### Task Overview

Step 1: Get daily and hourly historical Weather data for 5 specified coordinates

In [2]:
# Create a pandas dataframe with coordinates of locations to process
# Locations and coordinates found in openStreetMap
locations_coords = pd.DataFrame({
    "location" : ["Schönengrund, Hinterland, Appenzell Ausserrhoden, 9105, Switzerland", 
              "Einsiedeln, Schwyz, 8840, Switzerland",
              "Wolfenschiessen, Nidwalden, Switzerland",
              "Pontresina, Maloja, Grisons, 7504, Switzerland",
              "Ruvigliana, Circolo di Lugano est, Lugano, Distretto di Lugano, Ticino, 6977, Switzerland"
              ], 
    "lat" : [47.31706, 47.1071, 46.9062, 46.4774, 46.00865], 
    "lon" : [9.24482, 8.8114, 8.4282, 9.9042, 8.98767]})
print(locations_coords)


                                            location       lat      lon
0  Schönengrund, Hinterland, Appenzell Ausserrhod...  47.31706  9.24482
1              Einsiedeln, Schwyz, 8840, Switzerland  47.10710  8.81140
2            Wolfenschiessen, Nidwalden, Switzerland  46.90620  8.42820
3     Pontresina, Maloja, Grisons, 7504, Switzerland  46.47740  9.90420
4  Ruvigliana, Circolo di Lugano est, Lugano, Dis...  46.00865  8.98767


In [3]:
print(type(list(locations_coords.loc[:, "location"])))
print(list(locations_coords.loc[:, "location"]))

<class 'list'>
['Schönengrund, Hinterland, Appenzell Ausserrhoden, 9105, Switzerland', 'Einsiedeln, Schwyz, 8840, Switzerland', 'Wolfenschiessen, Nidwalden, Switzerland', 'Pontresina, Maloja, Grisons, 7504, Switzerland', 'Ruvigliana, Circolo di Lugano est, Lugano, Distretto di Lugano, Ticino, 6977, Switzerland']


In [23]:
# Define a start and end date of data to fetch from Open-Meteo API
start_date = "1941-01-01"
end_date = "2024-09-09"

In [24]:
# Setup the Open-Meteo API client with cache and retry on error
cache_session = requests_cache.CachedSession('.cache', expire_after = 3600)     # stores data for 1 hour in the cache
retry_session = retry(cache_session, retries = 5, backoff_factor = 0.2)         # retries 5 times with increasing delay of 20%
openmeteo = openmeteo_requests.Client(session = retry_session)                  # Using the retry_session to make a request to Open-Meteo API

In [25]:
# Make sure all required weather variables are listed here
# The order of variables in hourly or daily is important to assign them correctly below
url = "https://archive-api.open-meteo.com/v1/archive"
params = {
	"latitude": list(locations_coords["lat"]),
	"longitude": list(locations_coords["lon"]),
	"start_date": start_date,
	"end_date": end_date,
	"hourly": ["temperature_2m", "relative_humidity_2m", "dew_point_2m", "apparent_temperature", "precipitation", "rain", "snowfall", "snow_depth", "weather_code", "pressure_msl", "surface_pressure", "cloud_cover", "cloud_cover_low", "cloud_cover_mid", "cloud_cover_high", "wind_speed_10m", "wind_direction_10m", "wind_gusts_10m", "is_day", "sunshine_duration"],
	"daily": ["weather_code", "temperature_2m_max", "temperature_2m_min", "temperature_2m_mean", "apparent_temperature_max", "apparent_temperature_min", "apparent_temperature_mean", "sunrise", "sunset", "daylight_duration", "sunshine_duration", "precipitation_sum", "rain_sum", "snowfall_sum", "precipitation_hours", "wind_speed_10m_max", "wind_gusts_10m_max", "wind_direction_10m_dominant", "shortwave_radiation_sum"],
	"timezone": "Europe/Berlin"
}
responses = openmeteo.weather_api(url, params=params)
print(f"Number of resposes: {len(responses)}")

OpenMeteoRequestsError: {'reason': 'Hourly API request limit exceeded. Please try again in the next hour.', 'error': True}

In [18]:
# Process first location. Add a for-loop for multiple locations or weather models
response = responses[4]
print(f"Coordinates {response.Latitude()}°N {response.Longitude()}°E")
print(f"Elevation {response.Elevation()} m asl")
print(f"Timezone {response.Timezone()} {response.TimezoneAbbreviation()}")
print(f"Timezone difference to GMT+0 {response.UtcOffsetSeconds()} s")

Coordinates 46.01054382324219°N 9.142857551574707°E
Elevation 926.0 m asl
Timezone b'Europe/Berlin' b'CEST'
Timezone difference to GMT+0 7200 s


In [19]:
# Process hourly data. The order of variables needs to be the same as requested.
hourly = response.Hourly()
hourly_temperature_2m = hourly.Variables(0).ValuesAsNumpy()
hourly_relative_humidity_2m = hourly.Variables(1).ValuesAsNumpy()
hourly_dew_point_2m = hourly.Variables(2).ValuesAsNumpy()
hourly_apparent_temperature = hourly.Variables(3).ValuesAsNumpy()
hourly_precipitation = hourly.Variables(4).ValuesAsNumpy()
hourly_rain = hourly.Variables(5).ValuesAsNumpy()
hourly_snowfall = hourly.Variables(6).ValuesAsNumpy()
hourly_snow_depth = hourly.Variables(7).ValuesAsNumpy()
hourly_weather_code = hourly.Variables(8).ValuesAsNumpy()
hourly_pressure_msl = hourly.Variables(9).ValuesAsNumpy()
hourly_surface_pressure = hourly.Variables(10).ValuesAsNumpy()
hourly_cloud_cover = hourly.Variables(11).ValuesAsNumpy()
hourly_cloud_cover_low = hourly.Variables(12).ValuesAsNumpy()
hourly_cloud_cover_mid = hourly.Variables(13).ValuesAsNumpy()
hourly_cloud_cover_high = hourly.Variables(14).ValuesAsNumpy()
hourly_wind_speed_10m = hourly.Variables(15).ValuesAsNumpy()
hourly_wind_direction_10m = hourly.Variables(16).ValuesAsNumpy()
hourly_wind_gusts_10m = hourly.Variables(17).ValuesAsNumpy()
hourly_is_day = hourly.Variables(18).ValuesAsNumpy()
hourly_sunshine_duration = hourly.Variables(19).ValuesAsNumpy()

hourly_data = {"date": pd.date_range(
	start = pd.to_datetime(hourly.Time(), unit = "s", utc = True),
	end = pd.to_datetime(hourly.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = hourly.Interval()),
	inclusive = "left"
)}

hourly_data["lat"] = response.Latitude()
hourly_data["lon"] = response.Longitude()

hourly_data["temperature_2m"] = hourly_temperature_2m
hourly_data["relative_humidity_2m"] = hourly_relative_humidity_2m
hourly_data["dew_point_2m"] = hourly_dew_point_2m
hourly_data["apparent_temperature"] = hourly_apparent_temperature
hourly_data["precipitation"] = hourly_precipitation
hourly_data["rain"] = hourly_rain
hourly_data["snowfall"] = hourly_snowfall
hourly_data["snow_depth"] = hourly_snow_depth
hourly_data["weather_code"] = hourly_weather_code
hourly_data["pressure_msl"] = hourly_pressure_msl
hourly_data["surface_pressure"] = hourly_surface_pressure
hourly_data["cloud_cover"] = hourly_cloud_cover
hourly_data["cloud_cover_low"] = hourly_cloud_cover_low
hourly_data["cloud_cover_mid"] = hourly_cloud_cover_mid
hourly_data["cloud_cover_high"] = hourly_cloud_cover_high
hourly_data["wind_speed_10m"] = hourly_wind_speed_10m
hourly_data["wind_direction_10m"] = hourly_wind_direction_10m
hourly_data["wind_gusts_10m"] = hourly_wind_gusts_10m
hourly_data["is_day"] = hourly_is_day
hourly_data["sunshine_duration"] = hourly_sunshine_duration

hourly_dataframe = pd.DataFrame(data = hourly_data)
print(hourly_dataframe)

                            date        lat       lon  temperature_2m  \
0      1999-12-31 22:00:00+00:00  46.010544  9.142858         -2.6605   
1      1999-12-31 23:00:00+00:00  46.010544  9.142858         -2.6605   
2      2000-01-01 00:00:00+00:00  46.010544  9.142858         -2.8605   
3      2000-01-01 01:00:00+00:00  46.010544  9.142858         -3.6605   
4      2000-01-01 02:00:00+00:00  46.010544  9.142858         -4.7605   
...                          ...        ...       ...             ...   
216451 2024-09-09 17:00:00+00:00  46.010544  9.142858             NaN   
216452 2024-09-09 18:00:00+00:00  46.010544  9.142858             NaN   
216453 2024-09-09 19:00:00+00:00  46.010544  9.142858             NaN   
216454 2024-09-09 20:00:00+00:00  46.010544  9.142858             NaN   
216455 2024-09-09 21:00:00+00:00  46.010544  9.142858             NaN   

        relative_humidity_2m  dew_point_2m  apparent_temperature  \
0                  60.703217     -9.210501             

In [14]:
# Process daily data. The order of variables needs to be the same as requested.
daily = response.Daily()
daily_weather_code = daily.Variables(0).ValuesAsNumpy()
daily_temperature_2m_max = daily.Variables(1).ValuesAsNumpy()
daily_temperature_2m_min = daily.Variables(2).ValuesAsNumpy()
daily_temperature_2m_mean = daily.Variables(3).ValuesAsNumpy()
daily_apparent_temperature_max = daily.Variables(4).ValuesAsNumpy()
daily_apparent_temperature_min = daily.Variables(5).ValuesAsNumpy()
daily_apparent_temperature_mean = daily.Variables(6).ValuesAsNumpy()
daily_sunrise = daily.Variables(7).ValuesAsNumpy()
daily_sunset = daily.Variables(8).ValuesAsNumpy()
daily_daylight_duration = daily.Variables(9).ValuesAsNumpy()
daily_sunshine_duration = daily.Variables(10).ValuesAsNumpy()
daily_precipitation_sum = daily.Variables(11).ValuesAsNumpy()
daily_rain_sum = daily.Variables(12).ValuesAsNumpy()
daily_snowfall_sum = daily.Variables(13).ValuesAsNumpy()
daily_precipitation_hours = daily.Variables(14).ValuesAsNumpy()
daily_wind_speed_10m_max = daily.Variables(15).ValuesAsNumpy()
daily_wind_gusts_10m_max = daily.Variables(16).ValuesAsNumpy()
daily_wind_direction_10m_dominant = daily.Variables(17).ValuesAsNumpy()
daily_shortwave_radiation_sum = daily.Variables(18).ValuesAsNumpy()

daily_data = {"date": pd.date_range(
	start = pd.to_datetime(daily.Time(), unit = "s", utc = True),
	end = pd.to_datetime(daily.TimeEnd(), unit = "s", utc = True),
	freq = pd.Timedelta(seconds = daily.Interval()),
	inclusive = "left"
)}

daily_data["lat"] = response.Latitude()
daily_data["lon"] = response.Longitude()

daily_data["weather_code"] = daily_weather_code
daily_data["temperature_2m_max"] = daily_temperature_2m_max
daily_data["temperature_2m_min"] = daily_temperature_2m_min
daily_data["temperature_2m_mean"] = daily_temperature_2m_mean
daily_data["apparent_temperature_max"] = daily_apparent_temperature_max
daily_data["apparent_temperature_min"] = daily_apparent_temperature_min
daily_data["apparent_temperature_mean"] = daily_apparent_temperature_mean
daily_data["sunrise"] = daily_sunrise
daily_data["sunset"] = daily_sunset
daily_data["daylight_duration"] = daily_daylight_duration
daily_data["sunshine_duration"] = daily_sunshine_duration
daily_data["precipitation_sum"] = daily_precipitation_sum
daily_data["rain_sum"] = daily_rain_sum
daily_data["snowfall_sum"] = daily_snowfall_sum
daily_data["precipitation_hours"] = daily_precipitation_hours
daily_data["wind_speed_10m_max"] = daily_wind_speed_10m_max
daily_data["wind_gusts_10m_max"] = daily_wind_gusts_10m_max
daily_data["wind_direction_10m_dominant"] = daily_wind_direction_10m_dominant
daily_data["shortwave_radiation_sum"] = daily_shortwave_radiation_sum

daily_dataframe = pd.DataFrame(data = daily_data)
print(daily_dataframe)

                          date        lat       lon  weather_code  \
0    2019-12-31 22:00:00+00:00  46.010544  9.142858           0.0   
1    2020-01-01 22:00:00+00:00  46.010544  9.142858           0.0   
2    2020-01-02 22:00:00+00:00  46.010544  9.142858           2.0   
3    2020-01-03 22:00:00+00:00  46.010544  9.142858           1.0   
4    2020-01-04 22:00:00+00:00  46.010544  9.142858           0.0   
...                        ...        ...       ...           ...   
1709 2024-09-04 22:00:00+00:00  46.010544  9.142858          63.0   
1710 2024-09-05 22:00:00+00:00  46.010544  9.142858          51.0   
1711 2024-09-06 22:00:00+00:00  46.010544  9.142858          53.0   
1712 2024-09-07 22:00:00+00:00  46.010544  9.142858          51.0   
1713 2024-09-08 22:00:00+00:00  46.010544  9.142858           NaN   

      temperature_2m_max  temperature_2m_min  temperature_2m_mean  \
0               9.398500             -0.1515             3.890167   
1               7.748500         

In [60]:
# Save dataframes as csv files
hourly_dataframe.to_csv("historic_weather_hourly.csv")
daily_dataframe.to_csv("historic_weather_daily.csv")